# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list[:5])

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv']


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[:2])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056
[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner', 'Logged In', 'Mohammad', 'M', '0', 'Rodriguez', '277.15873', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', '200', '1.54328E+12', '88'], ['Jimi Hendrix', 'Logged In', 'Mohammad', 'M', '1', 'Rodriguez', '239.82975', 'paid', 'Sacramento--Roseville--Arden-Arcade, CA', 'PUT', 'NextSong', '1.54051E+12', '961', 'Woodstock Inprovisation', '200', '1.54328E+12', '88']]


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_library 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('music_library')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
query = """CREATE TABLE IF NOT EXISTS item_sessions
(
    session_id INT, 
    item_in_session INT, 
    artist TEXT, 
    song_title TEXT, 
    song_length FLOAT, 
    PRIMARY KEY (session_id, item_in_session)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    next(csvreader) # skip header
    
    for line in csvreader:

        query = """INSERT INTO item_sessions (session_id, item_in_session, artist, song_title, song_length)
                VALUES (%s, %s, %s, %s, %s)
                """

        session.execute(query, (int(line[-3]), int(line[3]), line[0], line[-2], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [10]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

query = """select artist, song_title, song_length from item_sessions
        WHERE session_id=%s AND item_in_session=%s
        """
try:
    rows = session.execute(query, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


In [11]:
query = """CREATE TABLE IF NOT EXISTS user_sessions
(
    user_id INT,
    session_id INT, 
    item_in_session INT, 
    artist TEXT,
    first_name TEXT, 
    last_name TEXT, 
    song_title TEXT,
    PRIMARY KEY (user_id, session_id, item_in_session)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [12]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    next(csvreader) # skip header
    
    for line in csvreader:

        query = """INSERT INTO user_sessions (user_id, session_id, item_in_session, artist, first_name, last_name, song_title)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
                """

        session.execute(query, (int(line[-1]), int(line[-3]), int(line[3]), line[0], line[1], line[5], line[-2]))

In [13]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query = """select artist, song_title, first_name, last_name from user_sessions
        WHERE user_id=%s AND session_id=%s
        """
try:
    rows = session.execute(query, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='333.76608')
Row(artist='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='411.6371')
Row(artist='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='377.73016')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='181.21098')


In [14]:
query = """CREATE TABLE IF NOT EXISTS user_songs
(
    user_id INT, 
    first_name TEXT, 
    last_name TEXT, 
    song_title TEXT,  
    PRIMARY KEY (song_title, user_id)
)
"""

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    
    next(csvreader) # skip header
    
    for line in csvreader:

        query = """INSERT INTO user_songs (user_id, first_name, last_name, song_title)
                VALUES (%s, %s, %s, %s)
                """

        session.execute(query, (int(line[-1]), line[1], line[4], line[-2]))

In [16]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query = """select first_name, last_name from user_songs
        WHERE song_title=%s
        """
try:
    rows = session.execute(query, ('All Hands Against His Own',))
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [60]:
session.execute("DROP TABLE IF EXISTS item_sessions")
session.execute("DROP TABLE IF EXISTS user_sessions")
session.execute("DROP TABLE IF EXISTS user_songs")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()